In [ ]:
# Nvidia Apex - Extension for using Pytorch
# Url - https://github.com/NVIDIA/apex

!cp -rp ../input/nvidiaapex/apex-master/* ./

!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./